Let's get all the transcripts first

In [196]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import numpy as np
import re
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import pandas as pd

# EDA for cleaning/scraping functions

- episode name
- episode number
- host
- contributers
- number of acts
- clean transcript
- approx episode length
- date originally published
- 

In [56]:
ep_number = '823'
transcript_url = 'https://www.thisamericanlife.org/'+ ep_number + '/transcript'
page = requests.get(transcript_url)

In [57]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
soup.select_one('h1')

<h1>823: The Question Trap</h1>

In [7]:
soup.select_one('body')

<body class="html not-front not-logged-in no-sidebars page-node page-node- page-node-45544 node-type-transcript page-status-">
<!-- Google Tag Manager (noscript) -->
<noscript><iframe height="0" src="https://www.googletagmanager.com/ns.html?id=GTM-5D6MWRV" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
<div id="skip-link">
<a class="element-invisible element-focusable" href="#content">Skip to main content</a>
</div>
<!--[if lt IE 7]>
   <p class="chromeframe">You are using an <strong>outdated</strong> browser. Please <a href="http://browsehappy.com/">upgrade your browser</a> or <a href="http://www.google.com/chromeframe/?redirect=true">activate Google Chrome Frame</a> to improve your experience.</p>
   <![endif]-->
<div aria-hidden="true" class="modal" data-micromodal-close="" id="takeover">
<div data-micromodal-close="" tabindex="-1">
<div aria-labelledby="modal-takeover-title" aria-modal="true" class="modal-container" 

In [8]:
# scraping and saving transcript html
'''for i in np.arange(1,824):
    ep_number = str(i)
    transcript_url = 'https://www.thisamericanlife.org/'+ ep_number + '/transcript'
    page = requests.get(transcript_url)
    with open('transcript-html/'+ep_number+'.html','wb+') as f: 
        f.write(page.content)'''

"for i in np.arange(1,824):\n    ep_number = str(i)\n    transcript_url = 'https://www.thisamericanlife.org/'+ ep_number + '/transcript'\n    page = requests.get(transcript_url)\n    with open('transcript-html/'+ep_number+'.html','wb+') as f: \n        f.write(page.content)"

In [10]:
# title tag gives us episode number and title
soup.title

<title>823: The Question Trap - This American Life</title>

In [17]:
tags = [tag.name for tag in soup.find_all()]
class_list = set()
for tag in tags: 
    for i in soup.find_all(tag):
        if i.has_attr("class"):
            if len(i['class']) !=0:
                class_list.add(" ".join(i['class']))

In [18]:
class_list

{'about has-children',
 'act',
 'act-inner',
 'actions',
 'apple',
 'block block-system',
 'body',
 'bottom',
 'button form-submit',
 'clear',
 'close',
 'close ignore',
 'column sidebar',
 'contact',
 'container clearfix',
 'content',
 'contributor',
 'copyright',
 'cta_link',
 'current',
 'element-invisible',
 'element-invisible element-focusable',
 'episode',
 'episodes',
 'facebook',
 'field-name-field-notes',
 'first',
 'flag-wrapper',
 'footer-inner clearfix',
 'footer-link last',
 'full-episode goto goto-episode',
 'host',
 'how-to-listen',
 'how-to-listen first',
 'html not-front not-logged-in no-sidebars page-node page-node- page-node-45544 node-type-transcript page-status-',
 'icon icon-arrow-left',
 'icon icon-close',
 'icon icon-closed_caption',
 'icon icon-facebook',
 'icon icon-flag',
 'icon icon-mail',
 'icon icon-pause',
 'icon icon-play',
 'icon icon-podcast',
 'icon icon-share',
 'icon icon-spotify',
 'icon icon-twitter',
 'icon icon-wbez',
 'icon icon-wordmark',
 'ic

In [178]:
soup.select('time')

[]

In [9]:
with open('transcript-text/'+str(824)+'.txt','w') as file:
    file.writelines(soup.get_text())

In [10]:
# for turning html into text transcripts 
'''for i in np.arange(1,824):
    with open('transcript-text/'+str(824)+'.txt','w') as file:
    file.writelines(soup.get_text())'''

"for i in np.arange(1,824):\n    with open('transcript-text/'+str(824)+'.txt','w') as file:\n    file.writelines(soup.get_text())"

In [23]:
soup.title

<title>823: The Question Trap - This American Life</title>

In [63]:
content = soup.find_all("div", class_="content")
csoup = BeautifulSoup(str(content))


In [68]:
csoup.get_text(strip=True)

'[,Full episodeToggle Audio and Transcript SyncTranscript823: The Question\xa0TrapNote: This American Life is produced for the ear and designed to be heard. If you are able, we strongly encourage you to listen to the audio, which includes emotion and emphasis that\'s not on the page. Transcripts are generated using a combination of speech recognition software and human transcribers, and may contain errors. Please check the corresponding audio before quoting in print.Prologue: PrologueIra GlassThe question started right after Tobin and his husband moved tothe Bay Area and got a house together.Tobin\'s family was pretty excited about this. They all live within an hour. And they brought meals over for weeks. His mom bought them shades. But this question popped up.Tobin  LowAnd the first time I noticed it happening, it was with my aunt. Kind of out of nowhere, she was like, oh, Which one of you is handy? Is one of you handy?And I was just like, why does she want to know that? Why does she 

In [76]:
soup.find("div", class_="host").h4.text #for host

'Ira Glass'

In [152]:
date = soup.find("meta", property="article:published_time")['content']

date

'2024-02-03T07:00:44-05:00'

In [167]:
from zoneinfo import ZoneInfo
dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
dt.astimezone(ZoneInfo('US/Pacific'))

datetime.datetime(2024, 2, 3, 4, 0, 44, tzinfo=zoneinfo.ZoneInfo(key='US/Pacific'))

In [181]:
soup.find_all('p')[289]

<p begin="00:55:12.24" data-timestamp="3312.24"> I'm Ira Glass. Back next week with more stories of This American Life.</p>

In [210]:
raw_ts = soup.findAll('p', {'begin':True})[287].get('begin')
ts = datetime.strptime(raw_ts, "%H:%M:%S.%f")
timestamp = timedelta(hours=ts.hour, minutes=ts.minute, seconds=ts.second)
timestamp

datetime.timedelta(seconds=3312)

In [214]:
meow = soup.findAll('p', {'begin':True})[287].get('data-timestamp')
pd.to_timedelta(float(meow), unit='s')

Timedelta('0 days 00:55:12.240000')

# HTML Scraping Functions

In [2]:
def clean_transcript(zup):
    content = zup.find_all("div", class_="content")
    c_soup = BeautifulSoup(str(content))
    return c_soup.get_text(strip=True)

In [1]:
def get_contributers(zup):
    contributers = set()
    contributers_raw = soup.select('.host h4')
    for n in contributers_raw:
        name = n.get_text(strip=True)
        name = re.sub("\\s+", " ", name)
        contributers.add(name)
    return list(contributers)

In [3]:
def get_episode(zup): 
    raw_title = zup.title.text
    split = re.split(":", raw_title, maxsplit=1)
    num = split[0]
    title = re.split("-",split[1])[0].strip()
    return num, title

In [4]:
def get_host(zup):
    return soup.find("div", class_="host").h4.text

In [5]:
def get_num_acts(zup): 
    return len(zup.find_all("div", class_='act')) - 2 #for prologue and credits

In [ ]:
def get_date(zup): 
    dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
    return dt.astimezone(ZoneInfo('US/Pacific'))

In [6]:
def get_last_timestamp(zup):
    num_ts = len(zup.find_all('p', {'begin':True}))
    raw_ts = zup.findAll('p', {'begin':True})[num_ts-1].get('data-timestamp')
    return pd.to_timedelta(float(raw_ts),unit='s')